<a href="https://colab.research.google.com/github/Purnima2004/ML-model-for-Energy-Conservation/blob/main/zelestra_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Preprocessing train data
df = pd.read_csv('train.csv')
df["power_output"] = df["voltage"] * df["current"]
df.dropna(subset=['power_output'], inplace=True)

features = df[["voltage", "current", "module_temperature", "cloud_coverage", "wind_speed", "pressure"]]
# Replace known bad values
features.replace(["badval", "", "N/A"], np.nan, inplace=True)

# Convert all to numeric
features = features.apply(pd.to_numeric, errors='coerce')

# Fill missing values (forward-fill using .ffill())
features.ffill(inplace=True)

# Scaling
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Function to create sequences for time series forecasting
def create_sequences(data, lookback=30):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i, :])
        y.append(data[i, -1])  # predicting the last feature ('pressure' here)
    return np.array(X), np.array(y)

# -------------------------------------------------
# Now process test data in exactly the same way!

test_df = pd.read_csv('test.csv')
test_df["power_output"] = test_df["voltage"] * test_df["current"]
test_df.dropna(subset=['power_output'], inplace=True)

test_features = test_df[["voltage", "current", "module_temperature", "cloud_coverage", "wind_speed", "pressure"]]
# Replace known bad values
test_features.replace(["badval", "", "N/A"], np.nan, inplace=True)

# Convert all to numeric
test_features = test_features.apply(pd.to_numeric, errors='coerce')

# Fill missing values (forward-fill)
test_features.ffill(inplace=True)

# Scaling using the SAME scaler
test_scaled_features = scaler.transform(test_features)

# Create sequences for test data (same lookback window)
X_test, y_test = create_sequences(test_scaled_features, lookback=30)


<ipython-input-5-518304331>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.replace(["badval", "", "N/A"], np.nan, inplace=True)
<ipython-input-5-518304331>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_features.replace(["badval", "", "N/A"], np.nan, inplace=True)


In [ ]:
#Training data
X,y= create_sequences(scaled_features)
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, shuffle=False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 0.0200 - val_loss: 0.0150
Epoch 2/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.0156 - val_loss: 0.0149
Epoch 3/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.0152 - val_loss: 0.0149
Epoch 4/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0150 - val_loss: 0.0152
Epoch 5/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 0.0149 - val_loss: 0.0149
Epoch 6/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0148 - val_loss: 0.0155
Epoch 7/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0148 - val_loss: 0.0150
Epoch 8/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0149 - val_loss: 0.0150
Epoch 9/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.0153 - val_loss: 0.0149
Epoch 10/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0149 - val_loss: 0.0153
Epoch 11/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0148 - val_loss: 0.0149
Epoch 12/30
903/903 ━━━━━━━━━━━━━━━━━━━━ 8s 8

In [ ]:
from sklearn.metrics import mean_squared_error
predicted= model.predict(X_test)
predicted= predicted.flatten()

mse= mean_squared_error(y_test, predicted)
print("MSE:", mse)

rmse= np.sqrt(mse)
print("RMSE:", rmse)

Score= 100*(1-rmse/np.mean(y_test))
print("Score:", Score)

113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
MSE: 0.01517502299117906
RMSE: 0.12318694326583098
Score: 76.15045424929544
